# Data url scraping
- using Selenium

In [25]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [252]:
class grab_urls:
    def __init__(self,url):
        self.url = url  
        self.opts = Options()
        self.opts.add_argument("--incognito")
        self.opts.headless = True
        self.driver = Chrome(options=self.opts)
        self.driver.get(self.url)
        

def url_grabber(driver):
    all_links = []
    while True:
        if 'page=2' in driver.current_url:
            try:
                linklist = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[*]/div[2]/div/a")
                links = [s.get_attribute('href') for s in linklist] 
                    
            except ElementClickInterceptedException:
                popup = driver.find_element_by_xpath("/html/body/div[12]/div/div[2]/div[2]/button[1]")
                popup.click()
                linklist = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[*]/div[2]/div/a")
                links = [s.get_attribute('href') for s in linklist] 
            all_links.append(links)

            page_lister = driver.find_element_by_class_name("category-page-list-related-nav-container")
            if len(page_lister.find_elements_by_tag_name("a"))==2: # 2 buttons(prev,next)
                next_page = page_lister.find_elements_by_tag_name("a")[1] #driver.find_element_by_xpath("/html/body/main/div[2]/div/div[29]/a[2]")
                next_page.click()
            else:
                return  all_links
                driver.quit()
                break


        else:
            driver.implicitly_wait(45)
            linklist = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div[*]/div[2]/div/a")
            links = [s.get_attribute('href') for s in linklist]
            all_links.append(links)
            page_lister = driver.find_element_by_class_name("category-page-list-related-nav-container")
            
            if len(page_lister.find_elements_by_tag_name("a"))==2: # 2 buttons(prev,next)
                next_page = page_lister.find_elements_by_tag_name("a")[1] #driver.find_element_by_xpath("/html/body/main/div[2]/div/div[29]/a[2]")
                next_page.click()
            else:
                driver.quit()
                break

    return  all_links        

In [256]:
url_list = [
"https://www.allrecipes.com/recipes/102/appetizers-and-snacks/antipasto/?page=2",
"https://www.allrecipes.com/recipes/14741/meat-and-poultry/pork/bacon/appetizers/?page=2",
"https://www.allrecipes.com/recipes/16833/appetizers-and-snacks/bruschetta/?page=2",
"https://www.allrecipes.com/recipes/17249/appetizers-and-snacks/canapes-and-crostini/?page=2",
"https://www.allrecipes.com/recipes/104/appetizers-and-snacks/cheese/?page=2",
"https://www.allrecipes.com/recipes/1240/appetizers-and-snacks/seafood/crab/crab-cakes/?page=2",
"https://www.allrecipes.com/recipes/16212/appetizers-and-snacks/snacks/crackers/?page=2",
"https://www.allrecipes.com/recipes/110/appetizers-and-snacks/deviled-eggs/?page=2",
"https://www.allrecipes.com/recipes/105/appetizers-and-snacks/dips-and-spreads/?page=2",
"https://www.allrecipes.com/recipes/1281/appetizers-and-snacks/dips-and-spreads/hummus/?page=2",
"https://www.allrecipes.com/recipes/436/appetizers-and-snacks/dips-and-spreads/salsa/?page=2",
"https://www.allrecipes.com/recipes/1244/appetizers-and-snacks/dips-and-spreads/shrimp-dip/?page=2",
"https://www.allrecipes.com/recipes/1242/appetizers-and-snacks/dips-and-spreads/spinach-dips/?page=2",
"https://www.allrecipes.com/recipes/1335/appetizers-and-snacks/dips-and-spreads/bean-dips/?page=2",
"https://www.allrecipes.com/recipes/13362/appetizers-and-snacks/dips-and-spreads/fruit-dip/?page=2",
"https://www.allrecipes.com/recipes/435/appetizers-and-snacks/dips-and-spreads/pate/?page=2",
"https://www.allrecipes.com/recipes/16068/breakfast-and-brunch/pancakes/whole-grain-pancakes/?page=2"
"https://www.allrecipes.com/recipes/16128/drinks/smoothies/banana/?page=2",
"https://www.allrecipes.com/recipes/16130/drinks/smoothies/blueberry/?page=2",
"https://www.allrecipes.com/recipes/17693/drinks/smoothies/green/?page=2",
"https://www.allrecipes.com/recipes/16131/drinks/smoothies/mango/?page=2",
"https://www.allrecipes.com/recipes/16129/drinks/smoothies/orange/?page=2",
"https://www.allrecipes.com/recipes/16127/drinks/smoothies/strawberry/?page=2",
"https://www.allrecipes.com/recipes/17578/drinks/smoothies/veggie/?page=2"
]

menu_names=[
"appetizers-and-snacks-file",
"breakfast-and-brunch-file",
"smoothies-file"
]

In [258]:
import time
t_start = time.time()
k0,k1,k2=0,0,0
for uri in url_list:
    if "appetizers-and-snacks" in uri:
        menu_name = f'{menu_names[0]}-{k0}'
        k0 += 1
    elif "breakfast-and-brunch" in uri:
        menu_name = f'{menu_names[1]}-{k1}'
        k1 += 1
    elif "smoothies" in uri:
        menu_name = f'{menu_names[2]}-{k2}'
        k2 += 1
    else:
        pass
        
    
    first_url = uri
    driver = grab_urls(first_url)
    result = url_grabber(driver.driver)
    df = pd.DataFrame(result).T
    names = [f'page-{i}' for i in range(2,len(result)+2)]
    df.columns = names
    # df.to_csv('chicken_urls.csv', index=False, header=True)
    df.to_csv(f'/home/zelalem/my_repos/mygithubrepos/vacation-projects/recipes/{menu_name}_urls.csv', index=False, header=True, mode = 'a')
    
t_end = time.time()
ellapsed_time = t_end - t_start
print('it took', ellapsed_time, 'seconds to scrap just the urls.')    

it took 51.3529417514801 seconds.


In [255]:
# Notice scraping has been in headless mode... Still can take a screenshot of the page ( for example to see what's going on when a click is not working) 
driver.driver.save_screenshot('screenshot99.png')

True

In [16]:
import glob
import itertools
import pandas as pd
files = ["appetizers-and-snacks-file-0_urls.csv","appetizers-and-snacks-file-1_urls.csv" ,
"appetizers-and-snacks-file-2_urls.csv","appetizers-and-snacks-file-3_urls.csv" ,
"appetizers-and-snacks-file-4_urls.csv","appetizers-and-snacks-file-5_urls.csv",
"appetizers-and-snacks-file-6_urls.csv","appetizers-and-snacks-file-7_urls.csv" ,
"breakfast-and-brunch-file-0_urls.csv","smoothies-file-0_urls.csv" ,
"smoothies-file-1_urls.csv","smoothies-file-2_urls.csv","smoothies-file-3_urls.csv" ,
"smoothies-file-4_urls.csv","smoothies-file-5_urls.csv","smoothies-file-6_urls.csv",
"smoothies-file-7_urls.csv","smoothies-file-8_urls.csv",
"smoothies-file-9_urls.csv","smoothies-file-10_urls.csv","smoothies-file-11_urls.csv",
"smoothies-file-12_urls.csv","smoothies-file-13_urls.csv", "dessert_urls.csv", "chicken_urls.csv"]

url_list_all = []
for file in files:
    # read urls scrapped using Selenium
    filename = glob.glob(f'/home/*/*/*/*/recipes/data/{file}')
    df = pd.read_csv(filename[0] , sep=",", engine='python', skiprows= 0, error_bad_lines=False)
    urls = [list(df[name]) for name in df] # make lists out of the df
    urls = list(itertools.chain(*urls)) # merge lists
    urls = list(filter(None, urls)) # remove NoneType values from list
    url_list_all.append(urls)
url_list_all = list(itertools.chain(*url_list_all)) # merge lists
url_list_all = list(filter(None, url_list_all)) # remove NoneType values from list
clean_urls = [x for x in url_list_all if x == x]  # remove nan's

print('Total number of urls to scraped using Selenium is {}'.format(len(clean_urls)))

Total number of urls to scraped using Selenium is 7464


In [15]:
# some recipes fall under multiple categories such as smoothies and appetizers - so need to clean duplicates before adding them to the database
from iteration_utilities import duplicates
print('Total number of duplicate urls is {}'.format(len(list(duplicates(clean_urls)))))


Total number of duplicate urls is 2335


In [17]:
# removing duplicates
clean_urls = list(set(clean_urls))
print('Total number of urls to be passed to scrapy is {}'.format(len(clean_urls)))

Total number of urls to be passed to scrapy is 5129


In [18]:
# save urls to csv file- will be read in by spider
pd.DataFrame({'urls': clean_urls}).to_csv('/home/zelalem/my_repos/mygithubrepos/vacation-projects/recipes/data/all_recipe_urls.csv', index=False, header=True, mode = 'w')

In [25]:
# read the the csv file for testing
filename = glob.glob(f'/home/*/*/*/*/recipes/data/all_recipe_urls.csv')
durl = pd.read_csv(filename[0] , sep=",", engine='python', skiprows= 0, error_bad_lines=False)
durl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5129 entries, 0 to 5128
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   urls    5129 non-null   object
dtypes: object(1)
memory usage: 40.2+ KB


In [23]:
durl.describe()

,urls
count,5129
unique,5129
top,https://www.allrecipes.com/recipe/25525/local-...
freq,1


In [29]:
urls_from_df = [list(durl[name]) for name in durl]
len(list(itertools.chain(*urls_from_df)))


5129